<a href="https://colab.research.google.com/github/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/MobileDet_Conversion_TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will see how to convert a pre-trained [MobileDet](https://arxiv.org/abs/2004.14525) model to TensorFlow Lite (using the latest converter). As the MobileDet model we are going to be using was trained in TensorFlow 1 we first need to generate a TensorFlow Lite compatible graph file following the instructions from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_on_mobile_tensorflowlite.md). 

In [1]:
#@title Initial setup
# Because MobileDet is based out of TensorFlow 1
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

!git clone --quiet https://github.com/tensorflow/models.git

TensorFlow 1.x selected.
1.15.2


In [ ]:
#@title Install the TFOD API
%%bash
pip install --upgrade pip
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

## Export to TFLite compatible graph

In [ ]:
#@title Fetch the checkpoints
#@markdown Model checkpoints are listed [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md).
mobiledet_checkpoint_name = "ssd_mobiledet_edgetpu_coco" #@param ["ssd_mobiledet_cpu_coco", "ssd_mobiledet_edgetpu_coco", "ssd_mobiledet_dsp_coco"]

checkpoint_dict = {
    "ssd_mobiledet_cpu_coco": "http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_cpu_320x320_coco_2020_05_19.tar.gz",
    "ssd_mobiledet_edgetpu_coco": "http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz",
    "ssd_mobiledet_dsp_coco": "http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_dsp_320x320_coco_2020_05_19.tar.gz"
}

folder_name_dict = {
    "ssd_mobiledet_cpu_coco": "ssdlite_mobiledet_cpu_320x320_coco_2020_05_19",
    "ssd_mobiledet_edgetpu_coco": "ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19",
    "ssd_mobiledet_dsp_coco": "ssdlite_mobiledet_dsp_320x320_coco_2020_05_19"
}

checkpoint_selected = checkpoint_dict[mobiledet_checkpoint_name]
folder_name = folder_name_dict[mobiledet_checkpoint_name]

# Get the pre-trained MobileDet checkpoints
!rm -rf folder_name
!wget -q $checkpoint_selected -O checkpoints.tar.gz
!tar -xvf checkpoints.tar.gz

In [ ]:
#@title Export a TFLite compatible model graph
#@markdown - For DSP & EdgeTPU provide `fp32` or `uint8` to the `trained_checkpoint_prefix` and `pipeline_config_path` arguments.

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$folder_name/fp32/pipeline.config \
    --trained_checkpoint_prefix=$folder_name//fp32/model.ckpt-400000 \
    --output_directory=`pwd` \
    --add_postprocessing_op=true

In [5]:
# Check the graph
!ls -lh *.pb

-rw-r--r-- 1 root root 17M Sep 11 06:59 tflite_graph.pb


We now need to switch to TensorFlow 2 runtime. We need to restart our Colab runtime first. 

## Export to TFLite

In [1]:
#@title Initial setup
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

2.3.0


As the `.pb` file we generated in the earlier step is a frozen graph, we need to use `tf.compat.v1.lite.TFLiteConverter.from_frozen_graph` to convert it to TFLite. 

In [2]:
#@title Quantize and serialize
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='tflite_graph.pb', 
    input_arrays=['normalized_input_image_tensor'],
    output_arrays=['raw_outputs/box_encodings', 'raw_outputs/class_predictions', 'anchors'],
    input_shapes={'normalized_input_image_tensor': [1, 320, 320, 3]}
)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('mobiledet.tflite', 'wb').write(tflite_model)
!ls -lh mobiledet.tflite

-rw-r--r-- 1 root root 4.2M Sep 11 07:00 mobiledet.tflite
